---
title: Accessing Zarr Data
subtitle: Explore how to access Data in Zarr format.
authors:
  - name: Simon Mosonyi
    github: SimonMosonyi
    affiliations:
      - id: eodc
        institution: eodc
date: 2025-09-25
thumbnail: ../static/ESA_EOPF_logo_2025_COLOR_ESA_blue_reduced.png
keywords: ["xarray", "stac"]
tags: ["xarray", "stac"]
releaseDate: 2025-09-25
datePublished: 2025-09-25
dateModified: 2025-09-25
github: https://github.com/EOPF-Sample-Service/eopf-sample-notebooks
license: Apache-2.0
---

```{image} ../static/ESA_EOPF_logo_2025_COLOR_ESA_blue_reduced.png
:alt: ESA EOPF Zarr Logo
:width: 250px
:align: center
```

## Table of Contents
1. [Introduction](#Introduction_STAC_TUTORIAL)
1. [Explore Data in STAC Browser](#explore_STAC_TUTORIAL)
2. [Access the Data](#access_STAC_TUTORIAL)
    - [Using `s5cmd`](#using_s5_cmd_STAC_TUTORIAL)
        - [Install `s5cmd`](#install_s5cmd_STAC_TUTORIAL)
        - [List Files in a Bucket](#list_STAC_TUTORIAL)
        - [Download Files Locally](#download_STAC_TUTORIAL)
    - [Using Python](#using_python_STAC_TUTORIAL)
        - [`pystac-client`](#pystac_client_STAC_TUTORIAL)
        - [`boto3`](#boto3_STAC_TUTORIAL)

(Introduction_STAC_TUTORIAL)=
## Introduction

This notebook demonstrates how to access Zarr-formatted data using:

- ```s5cmd```: A fast command-line tool for working with S3 object storage.
- Python libraries like ```pystac-client```, ```xarray```, and ```boto3```.

:::{hint} Overview
**Questions**
- What is the most efficient way to interact with Zarr-formatted Earth Observation data in the cloud?
- How can I explore available Sentinel datasets before downloading or loading them?
- What are the differences between using command-line tools and Python APIs to access Zarr data?

**Objectives**
- Explore Sentinel data collections through the STAC Browser and Python-based STAC API clients.
- Use ```s5cmd``` to list and download Zarr datasets from cloud object storage.
- Load Zarr data directly into Python using ```pystac-client```, ```boto3```, and ```xarray``` for scalable analysis.
:::

(explore_STAC_TUTORIAL)=
## Explore Data in STAC Browser

You can browse the available Sentinel datasets interactively using our [STAC Browser](https://stac.browser.user.eopf.eodc.eu/?.language=de). Use it to filter by collection, date, or location before programmatically accessing data.

(access_STAC_TUTORIAL)=
## Access the Data

Explore, interact with and retrieve Sentinel 1, 2 and 3 data in EOPF Zarr format.

**Direct download is not the recommended approach. We encourage users to interact with the data directly through cloud-based resources for better efficiency and scalability.**

(using_s5_cmd_STAC_TUTORIAL)=
### Using ```s5cmd```

> ```s5cmd``` is a very fast S3 and local filesystem execution tool. It comes with support for a multitude of operations including tab completion and wildcard support for files, which can be very handy for your object storage workflow while working with large number of files.

(install_s5cmd_STAC_TUTORIAL)=
#### Install ```s5cmd```

You can install ```s5cmd``` from the official [GitHub repository](https://github.com/peak/s5cmd?tab=readme-ov-file#installation).


(list_STAC_TUTORIAL)=
#### List Files in a bucket

```bash
s5cmd --endpoint-url https://objects.eodc.eu -no-sign-request ls \
"s3://e05ab01a9d56408d82ac32d69a5aae2a:202507-s03olclfr/*"
```

(download_STAC_TUTORIA)=
#### Download Files Locally

```bash
s5cmd --endpoint-url https://objects.eodc.eu -no-sign-request cp \
"s3://e05ab01a9d56408d82ac32d69a5aae2a:202507-s03olclfr/06/../products/cpm_v256/S3A_OL_2_LFR____20250706T083947_20250706T084247_20250707T125430_0179_128_007_2160_PS1_O_NT_003.zarr/*" \
./dummy.zarr/
```

(using_python_STAC_TUTORIAL)=
### Using Python

You can programmatically discover and load data with Python using the following libraries:

(pystac_client_STAC_TUTORIAL)=
#### pystac-client

Search for available data and load it directly from object storage using ```pystac-client``` and ```xarray```.

In [1]:
from pystac_client import Client

catalog = Client.open("https://stac.core.eopf.eodc.eu")
results = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=[7.2, 44.5, 7.4, 44.7],
    datetime=["2025-04-30", "2025-05-01"],
)

items = results.item_collection()[0]

Items found can be directly loaded with the ```xarray``` library.

In [ ]:
import xarray as xr

ds = xr.open_datatree(
    items.assets["product"].href + "/measurements",
    engine="zarr",
    chunks={},
    decode_timedelta=True,
    consolidated=False,
)
ds

(boto3_STAC_TUTORIAL)=
#### boto3

If you prefer working directly with the S3 API, use ```boto3``` to list and access objects.

EOPF products are organised on the object storage per month and product, depicted by the following bucket naming convention.

```e05ab01a9d56408d82ac32d69a5aae2a:<year><month>-<product-id>```

In the provided code snipped this results in the bucket name:

```e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil1c```

Furthermore, data is organised by day and the given software version used for conversion as indicated in the “prefix” variable in code.



In [ ]:
import boto3
from botocore.config import Config
from botocore import UNSIGNED
import botocore.handlers
import re

botocore.handlers.VALID_BUCKET = re.compile(r"^[:a-zA-Z0-9.\-_]{1,255}$")

# Setup
endpoint_url = "https://objects.eodc.eu"
tenant = "e05ab01a9d56408d82ac32d69a5aae2a"

s3_client = boto3.client(
    "s3", endpoint_url=endpoint_url, config=Config(signature_version=UNSIGNED)
)

# Define bucket and prefix
bucket = f"{tenant}:202506-s02msil1c"
prefix = "20/products/cpm_v256/"

# List objects under the prefix
response = s3_client.list_objects_v2(
    Bucket=bucket,
    Prefix=prefix,
)

# Print object keys
for obj in response.get("Contents", []):
    key = obj["Key"]
    match = re.search(r"\.zarr/", key)
    if match:
        store_root = key[: match.end() - 1]
        url = f"{endpoint_url}/{bucket}/{store_root}"
        print(url)

You can then load the ```.zarr```. dataset using the same approach as shown earlier with ```.xarray.open_datatree()```..